In [2]:
import pandas as pd
import sqlite3
import joblib
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [5]:
# Load the trained models
log_model = joblib.load("../Resources/logistic_regression_model.pkl")
tree_model = joblib.load("../Resources/decision_tree_model.pkl")

print("Models loaded successfully!")

Models loaded successfully!


In [7]:
# Connect to SQLite database
sqlite_db_path = "../student_performance.db"
conn = sqlite3.connect(sqlite_db_path)

df = pd.read_sql("SELECT * FROM student_mat", conn)
conn.close()

# Display first few rows
print("Sample Data:")
display(df.head())

Sample Data:


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [8]:
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

scaler = StandardScaler()
grade_cols = ['G1', 'G2', 'G3']
numeric_cols = [col for col in df.select_dtypes(include=['int64', 'float64']).columns if col not in grade_cols]
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

In [9]:
X_analysis = df.drop(columns=['G3'])

df['Predicted_Performance_Log'] = log_model.predict(X_analysis)
df['Predicted_Performance_Tree'] = tree_model.predict(X_analysis)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- attendance_engagement
- low_engagement_flag
- sudden_grade_drop


In [ ]:
# Displaying predictions
print("Predictions Sample:")
display(df[['G1', 'G2', 'G3', 'Predicted_Performance_Log', 'Predicted_Performance_Tree']].head())